In [102]:
!pip install --quiet graphviz
!pip install --quiet d3graph
!pip install --quiet treelib
!pip install --quiet tkinter
!pip install --quiet pixiedust_node
!pip install --quiet pyvis
!pip install --quiet netgraph
!pip install --quiet ipycytoscape

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [103]:
import os
import re
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import Checkbox, Text

<h1>clean(s)</h1>
<p>
    Removes spaces and symbols from strings to make them into valid identifiers.
</p>

In [104]:
def clean(s): return re.sub('\W|^(?=\d)','_', s).lower()

<h1>neat_trim(s, length)</h1>
<p>Trims a string to specified length, but tries to do it neatly at a word boundary.

In [105]:
def neat_trim(s, ideal_length, max_length):
    # Shortcut for strings that are split by an emdash.
    clauses = re.split(' - |[(:]', s)
    if len(clauses[0]) > max_length/2 and len(clauses[0]) < max_length: return clauses[0]
    
    # If the string is short enough, return it unmodified.
    if len(s) < max_length: return s
    
    # Split the string at the ideal length.
    front = s[:ideal_length]
    back = s[ideal_length:]
    
    # If there's nothing in the back half, return the front.
    # This should never happen!
    if back == None or back.isspace(): return front
    
    # Split the string into words.
    words = back.split(' ')
    
    # The length of the word that can be added to the end without exceeding max_length.
    grace = max_length - ideal_length - 1
        
    #print('Front = "' + front + '", words = ' + str(words))
    
    # If the split didn't produce anything, return the front.
    # This should never happen!
    if words == None or len(words) == 0: return front

    # If the first word is too long, return the front with an ellipsis ...
    elif len(words[0]) > grace: return front + '\u2026'
    
    # Everything is fine so return the front with the first appended.
    else: return f'{front}{words[0]}'

<h1>Utility functions for make_animations_dataframe</h1>

In [106]:
dir_path = None

def get_path():
    import tkinter as tk
    from tkinter import filedialog
    global dir_path
    if dir_path == None:
        tk.Tk().withdraw()
        dir_path = filedialog.askdirectory()
    return dir_path

def find_mod_name(path):
    if path.isspace() or not os.path.isdir(path):
        raise Exception("Not Found")
        
    tail,head = os.path.split(path)
    if head=='mods': return 'mods'
    
    parent = find_mod_name(tail)
    #print('\t\tParent returned: ', parent)
    if parent == 'mods': return head
    else: return parent

<h1>Function: make_animations_dataframe</h1>
This scans an MO2 directory looking for DAR animations.
It turns a 5-tuple consisting of: 
<ol>
    <li>mapping of animation->priority->mod
    <li>dataframe of all (animation,priority,mod) triplets
    <li>map of priorities to mods
    <li>map from priorities to lists of colliding mods
    <li>map from lists of colliding mods to list of colliding priorities
</ol>

In [144]:
"""
**animations**
priority -> animation filename -> mod
int -> string -> string

**priorities**
priority -> mod
int -> string

**priority_collisions**
priority -> set of mods
int -> set

**mod_collisions**
mod,mod -> list of priorities
list(string) -> list(int)

"""
def make_animations_dataframe():
    import glob
    from collections import defaultdict
    
    df = pd.DataFrame(columns=['Mod', 'Priority', 'Animation'])
    animations = {}
    priorities = {}
    #priority_collisions = {}
    priority_collisions = defaultdict(set)
    mod_collisions = {}

    for root, d_names, f_names in os.walk(get_path()):   
        path = os.path.split(root)
        directory_name = path[-1] 

        if directory_name.lower() == '_customconditions':
            #flagged = 'tk' in root.lower()
            flagged = False
            mod_name = 'invalid'        
            try:
                mod_name = find_mod_name(root).lower()
            except Exception as ex:
                print(path, ex)
                continue

            d_names_numeric = [d_name for d_name in d_names if d_name.isnumeric()]        
            if flagged: print(root, '\n', d_names_numeric)

            #print(mod_name)
            #print('\t', d_names_numeric[:5])

            for d_name in d_names_numeric:
                priority = int(d_name)
                group_path = os.path.join(root, d_name)
                animation_glob = os.path.join(group_path, '**', '*.hkx')
                condition_filename = os.path.join(group_path, '_conditions.txt')

                if priority not in priorities.keys(): priorities[priority] = mod_name
                elif priorities[priority] != mod_name:
                    #if priority not in priority_collisions.keys():
                    #    priority_collisions[priority] = set()
                    priority_collisions[priority].add(priorities[priority])                        
                    priority_collisions[priority].add(mod_name)
                    assert len(priority_collisions[priority]) > 1, f'Problem with {priority}, {mod_name}, {priorities[priority]}'
                    
                for anim in glob.glob(animation_glob, recursive=True):
                    anim_path,anim_name = os.path.split(anim)
                    relpath = os.path.relpath(anim_path, group_path)

                    if relpath != '.': anim_name = os.path.join(relpath, anim_name)

                    if flagged:
                        print('\t', mod_name, d_name, relpath, anim_name)

                    anim_name = anim_name.lower()
                    
                    if anim_name not in animations.keys():
                        animations[anim_name] = {}

                    if priority not in animations[anim_name].keys():
                        animations[anim_name][priority] = set()

                    animations[anim_name][priority].add(mod_name)

                    row = {'Mod':mod_name.lower(), 'Priority':priority, 'Animation':anim_name.lower()}
                    new_df = pd.DataFrame([row])
                    df = pd.concat([df, new_df], axis=0, ignore_index=True)
    
    for priority in priority_collisions:
        mods = tuple(sorted(priority_collisions[priority]))
        if mods not in mod_collisions.keys():
            mod_collisions[mods] = set()
        mod_collisions[mods].add(priority)

    return animations, df, priorities, priority_collisions, mod_collisions

In [108]:
def print_summary(data):
    animations, df, priorities, priority_collisions, mod_collisions = data
    print(f"{len(df)} DAR animations found (out of either 16384 or 32768, depending on version)")
    print(f"{len(priority_collisions)} priority collisions were found.")
    print(make_priorities_collisions_tree(mod_collisions))


In [109]:
def test_make_animations_dataframe():
    animations, df, priorities, priority_collisions, mod_collisions = make_animations_dataframe()
    return df.head()
#test_make_animations_dataframe()

<h1>TreeLib</h1>
<p>Display textual trees</p>

In [110]:
def print_priority_collisions(priority_collisions):
    for priority in priority_collisions:
        print(priority)
        print('\t', priority_collisions[priority])
        
def print_mod_collisions(mod_collisions):
    for mods in mod_collisions:
        print(mods)
        print('\t', mod_collisions[mods])

def make_priorities_collisions_tree(mod_collisions):
    from treelib import Node, Tree
    priorities_collisions_tree = Tree()
    priorities_collisions_root = priorities_collisions_tree.create_node('Priority Collisions', 'priority collisions')

    idv = 0
    def next_id():
        nonlocal idv
        idv += 1
        return str(idv)

    priorities_collisions = {tuple(priorities):tuple(mods) for (mods,priorities) in mod_collisions.items()}
    
    for priorities in priorities_collisions:
        mods = priorities_collisions[priorities]

        collision_node = next_id()
        priorities_collisions_tree.create_node('Collision', collision_node, 'priority collisions')        
        
        priorities_node = next_id()
        priorities_collisions_tree.create_node('Of priorities', priorities_node, collision_node)        
        
        for priority in priorities:
            priorities_collisions_tree.create_node(str(priority), next_id(), priorities_node)
            
        mods_node = next_id()
        priorities_collisions_tree.create_node('Of mods', mods_node, collision_node)        
            
        for mod in mods:
            priorities_collisions_tree.create_node(mod, next_id(), mods_node)
                        
    return priorities_collisions_tree


def make_mod_collisions_tree(mod_collisions):
    from treelib import Node, Tree
    mod_collisions_tree = Tree()
    mod_collisions_root = mod_collisions_tree.create_node('Mod Collisions', 'mod collisions')

    idv = 0
    def next_id():
        nonlocal idv
        idv += 1
        return str(idv)

    for k in mod_collisions:
        mods = set(k)
        mod_desc = '<' + '\n\t'.join(mods) + '>'

        collision_node = next_id()
        mod_collisions_tree.create_node('Collision', collision_node, 'mod collisions')        
        
        list_node = next_id()
        mod_collisions_tree.create_node('Between', list_node, collision_node)        
        for mod in mods:
            mod_collisions_tree.create_node(mod, next_id(), list_node)
            
        priorities_node = next_id()
        mod_collisions_tree.create_node('Priorities', priorities_node, collision_node)        
        for priority in mod_collisions[k]:
            priority_id = str(priority)
            mod_collisions_tree.create_node(priority_id, priority_id, priorities_node)
            
    return mod_collisions_tree

def make_anim_collisions_tree(animations):
    from treelib import Node, Tree
    anim_collisions_tree = Tree()
    anim_collisions_root = anim_collisions_tree.create_node('Animation Collisions', 'anim collisions')

    idv = 0
    def next_id():
        nonlocal idv
        idv += 1
        return str(idv)

    for anim_name in animations:    
        anim_id = str(anim_name)
        anim_collisions_tree.create_node(f'Animation: {anim_name}', anim_id, 'anim collisions')

        priorities = animations[anim_name]
        for priority in priorities:
            mods = priorities[priority]
            if len(mods) > 1:
                priority_id = anim_id + str(int(priority))
                anim_collisions_tree.create_node(priority, priority_id, anim_id)
                
                for mod in mods:
                    mod_id = priority_id + '_' + next_id()
                    anim_collisions_tree.create_node(mod, mod_id, priority_id)
                
    return anim_collisions_tree


<h1>PyVis Network</h1>

<p><a href='https://pyvis.readthedocs.io/en/latest/tutorial.html#visualization'>
    https://pyvis.readthedocs.io/en/latest/tutorial.html#visualization
</a></p>

<p><a href='https://visjs.github.io/vis-network/docs/network/nodes.html'>
    https://visjs.github.io/vis-network/docs/network/nodes.html
</a></p>

In [151]:
def pyvis_d3_force(data, only_conflicts=True, mod_kwd=None, anim_kwd=None):
    from pyvis.network import Network
    from collections import defaultdict
    
    animations, df, priorities, priority_collisions, mod_collisions = data

    pvnet = Network(1200,800, notebook=True)
    pvnet.toggle_hide_edges_on_drag(False)
    pvnet.barnes_hut()
    pvnet.toggle_physics(True)

    idv = 0
    def next_id():
        nonlocal idv
        idv += 1
        return idv
    
    node_ids = defaultdict(next_id)
    
    for animation in animations:
        if anim_kwd == None or anim_kwd.isspace() or anim_kwd in animation:
            anim_id = node_ids[animation]
            made_anim_node = False
            
            priorities = animations[animation]
            winning_priority = None
            
            for priority in priorities:
                if not only_conflicts or priority in priority_collisions.keys():
                    if not made_anim_node:
                        made_anim_node = True
                        pvnet.add_node(anim_id, label=animation, title=animation, size=30, color='blue', font='120px arial')
                    
                    priority_id = node_ids[priority]
                    #pvnet.add_node(priority_id, label=str(priority), title=str(priority), size=20, color='green', font='120px arial')
                    #pvnet.add_edge(anim_id, priority_id)
                    
                    mods = priorities[priority]
                    for mod in mods:
                        if mod_kwd == None or mod_kwd.isspace() or mod_kwd in mod:
                            if winning_priority == None: winning_priority = max(priorities.keys())
                            mod_id = node_ids[mod]
                            label = neat_trim(mod,20,25)
                            pvnet.add_node(mod_id, label=label, title=mod, size=50, color='red', mass=2, shape='dot', font='150px arial')
                            if priority == winning_priority:
                                pvnet.add_edge(anim_id, mod_id, title=priority, value = 1, font='100px arial')
                            else:
                                pvnet.add_edge(anim_id, mod_id, title=priority, value = 1, font='100px arial')

    return pvnet.show('pyvis_d3_force.html')


<h1>GraphVis</h1>
NOT WORKING

In [112]:
def graphviz_demo():
    from graphviz import Digraph

    def visualize_tree(tree):
        def add_nodes_edges(tree, dot=None):
            # Create Digraph object
            if dot is None:
                dot = Digraph()
                dot.node(name=str(tree), label=str(tree.val))

            # Add nodes
            if tree.left:
                dot.node(name=str(tree.left) ,label=str(tree.left.val))
                dot.edge(str(tree), str(tree.left))
                dot = add_nodes_edges(tree.left, dot=dot)

            if tree.right:
                dot.node(name=str(tree.right) ,label=str(tree.right.val))
                dot.edge(str(tree), str(tree.right))
                dot = add_nodes_edges(tree.right, dot=dot)

            return dot

        # Add nodes recursively and create a list of edges
        dot = add_nodes_edges(tree)

        # Visualize the graph
        display(dot)

        return dot

    dot = Digraph()
    # Add nodes 1 and 2
    dot.node('1')
    dot.node('2')

    # Add edge between 1 and 2
    dot.edges(['12'])
    # Visualize the graph
    return dot


<h1>Utility functions for Dash display_treemap</h1>

In [113]:
def filter_dataframe(df, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    dff = df

    if only_conflicts:
        df_anim_counts = df.Animation.value_counts()
        anim_filter = dff.Animation.isin(df_anim_counts.index[df_anim_counts.gt(1)])
        dff = dff[anim_filter]
    
    if anim_kwd != None and not anim_kwd.isspace():
        #print('Filtering: show only animations that contain', anim_filter)
        dff = dff[dff.Animation.str.contains(anim_kwd)]

    if (mod_kwd != None) and not mod_kwd.isspace():
        #print('Filtering: show only mods that contain', mod_filter)
        dff = dff[dff.Mod.str.contains(mod_kwd)]
        
    return dff

def make_treemap(df_filtered, path=None):
    import plotly.express as px
    if path == None: path = [px.Constant("<br>"), 'Animation', 'Mod', 'Priority']
    elif path == 'org1': path = [px.Constant("<br>"), 'Animation', 'Mod', 'Priority']
    elif path == 'org2': path = [px.Constant("<br>"), 'Animation', 'Priority', 'Mod']
    elif path == 'org3': path = [px.Constant("<br>"), 'Mod', 'Animation', 'Priority']
    elif path == 'org4': path = [px.Constant("<br>"), 'Mod', 'Priority', 'Animation']
    elif path == 'org5': path = [px.Constant("<br>"), 'Priority', 'Mod', 'Animation']
    elif path == 'org6': path = [px.Constant("<br>"), 'Priority', 'Animation', 'Mod']
    elif path == 'org7': 
        path = [px.Constant("<br>"), 'Priority', 'Mod']
        df_filtered = df_filtered.drop('Animation', axis=1).drop_duplicates().sort_values(by=['Priority'])
        df_priority_counts = df_filtered.Priority.value_counts()
        filt = df_filtered.Priority.isin(df_priority_counts.index[df_priority_counts.gt(1)])
        df_filtered = df_filtered[filt]
        
    else: path = [px.Constant("<br>"), 'Animation', 'Mod', 'Priority']
    
    fig = px.treemap(df_filtered, path=path)
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    return fig


<h1>Dash: display_treemap</h1>
Display a Dash TreeMap with filter controls.

In [114]:
def display_treemap(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    animations, df, priorities, priority_collisions, mod_collisions = data
    
    import plotly.graph_objects as go
    import dash
    from dash import html, dcc, Input, Output
    from jupyter_dash import JupyterDash

    app = JupyterDash(__name__)

    fig = make_treemap(filter_dataframe(df, only_conflicts=True, anim_kwd=anim_kwd, mod_kwd=mod_kwd))

    app.layout = html.Div(children=[
            html.Div([
                html.Div([
                        html.P('Animation Filter Text:'),
                        dcc.Input(id='filter_animations', value=anim_kwd, placeholder='(animation name)'),
                ], style={'display': 'inline-block'}),
                html.Div(style={'width':'3%', 'display': 'inline-block'}),
                html.Div([
                    html.P('Mod Filter Text:'),
                    dcc.Input(id='filter_mods', value=mod_kwd, placeholder='(animation name)'),
                ], style={'display': 'inline-block'}),
            ]),
            html.Div([
                #dcc.Graph(id='force_graph')
            ]),
            html.Div([
                html.Div([
                        dcc.Checklist(
                            id='filter_conflicts',  
                            options={'conflicts':'Show only conflicts'},
                            value=['conflicts']),
                ], style={'display': 'inline-block'}),
                html.Div(style={'width':'3%', 'display': 'inline-block'}),
                html.Div(
                    dcc.Dropdown(id='organization', options={
                        'org1':'Animation->Mod->Priority',
                        'org2':'Animation->Priority->Mod',
                        'org3':'Mod->Animation->Priority',
                        'org4':'Mod->Priority->Animation',
                        'org5':'Priority->Mod->Animation',
                        'org6':'Priority->Animation->Mod',
                        'org7':'Priority->Mod',
                    }, value='org7', placeholder='This is a long string', style={'width':'300px'}), style={'display': 'inline-block'}),
                
                dcc.Graph(id='figure_treemap', figure=fig)
            ], id='figure1_div'),
        ])

    @app.callback(
        Output(component_id='figure_treemap', component_property='figure'),
        Input(component_id='filter_animations', component_property='value'),
        Input(component_id='filter_mods', component_property='value'),
        Input(component_id='filter_conflicts', component_property='value'),
        Input(component_id='organization', component_property='value'),
    )
    def update_tree_amp(anim_kwd, mod_kwd, filter_conflicts, organization):
        only_conflicts = filter_conflicts == 'conflicts'
        df2 = filter_dataframe(df, only_conflicts=only_conflicts, anim_kwd=anim_kwd, mod_kwd=mod_kwd)
        return make_treemap(df2, organization)

    app.run_server(mode="inline")


<h1>External D3 HTML/CSS/JavaScript: make_pcap_json</h1>
<p>
    Based on 
    <a href='http://www.austintaylor.io/d3/python/pandas/2016/02/01/create-d3-chart-python-force-directed/'>
        http://www.austintaylor.io/d3/python/pandas/2016/02/01/create-d3-chart-python-force-directed/
    </a>
</p>

<p>
    Requires running <code>python -m http.server</code> to host the page.
</p>

In [116]:
def make_pcap_json(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    import json
    import webbrowser
    
    animations, df, priorities, priority_collisions, mod_collisions = data
    if anim_kwd != None and not anim_kwd.isspace():
        df = df[df.Animation.str.contains(anim_kwd)]
    if mod_kwd != None and not mod_kwd.isspace():
        df = df[df.Mod.str.contains(mod_kwd)]
    if only_conflicts:
        df = df[df.Priority.inin(priority_collisions.keys())]
        
    nodes1 = list(df.Mod.drop_duplicates())
    nodes2 = list(df.Priority.drop_duplicates())
    nodes3 = list(df.Animation.drop_duplicates())

    #print(nodes2[:50])

    src_dest1 = df[['Mod', 'Priority']]
    src_dest2 = df[['Mod', 'Animation']]
    src_dest3 = df[['Priority', 'Animation']]

    src_dest1.columns = ['source', 'target']
    src_dest2.columns = ['source', 'target']
    src_dest3.columns = ['source', 'target']
    src_dest = pd.concat([src_dest1, src_dest2, src_dest3])

    grouped_src_dst = src_dest.groupby(["source","target"]).size().reset_index()
    grouped_src_dst.rename(columns={0:'count'}, inplace=True)
    #print(grouped_src_dst.head())

    temp_links_list = list(grouped_src_dst.apply(lambda row: {"source": row['source'], "target": row['target'], "value": row['count']}, axis=1))

    unique_nodes = pd.Index(pd.concat([grouped_src_dst['source'], grouped_src_dst['target']]).reset_index(drop=True).unique())
    #print(unique_nodes)

    links_list = []
    for link in temp_links_list:
        record = {"value":link['value'], "source":unique_nodes.get_loc(link['source']),
         "target": unique_nodes.get_loc(link['target'])}
        links_list.append(record)


    nodes_list = []

    for node in unique_nodes:
        group = 0
        if node in nodes1: nodes_list.append({'group':'Mod', 'name':node})
        elif node in nodes2: nodes_list.append({'group':'Priority', 'name':node})
        elif node in nodes3: nodes_list.append({'group':'Animation', 'name':node})
        #print(nodes_list[-1])


    json_prep = {"nodes":nodes_list, "links":links_list}
    json_dump = json.dumps(json_prep, indent=1, sort_keys=True)
    #print(json_dump)

    with open('pcap_export.json','w') as json_out:
        json_out.write(json_dump)
        
    webbrowser.open('http://localhost:8000/index.html')


<h1>D3Graph: simple_force_graph</h1>
Displays in a new webpage.

In [117]:
def simple_force_graph(data, explicit_priorities=False, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    from d3graph import d3graph, vec2adjmat
    
    animations, df, priorities, priority_collisions, mod_collisions = data

    source = []
    target = []
    weight = []

    for anim in animations:
        if anim_kwd == None or anim_kwd.isspace() or anim_kwd in anim:
            priorities = animations[anim]
            for priority in priorities:
                if not only_conflicts or priority in priority_collisions.keys():
                    mods = priorities[priority]

                    if explicit_priorities:
                        source.append(priority)
                        target.append(anim)
                        weight.append(len(mods))

                    for mod in mods:
                        if mod_kwd == None or mod_kwd.isspace() or mod_kwd in mod:
                            if explicit_priorities:
                                source.append(mod)
                                target.append(priority)
                                weight.append(1)
                            else:
                                source.append(mod)
                                target.append(anim)
                                weight.append(1)
                                

    source = list(source)
    target = list(target)
    weight = list(weight)

    adjmat = vec2adjmat(source, target, weight=weight)
    # Initialize
    d3 = d3graph()

    # Proces adjmat
    d3.graph(adjmat)
    
    # Plot
    d3.set_edge_properties(edge_distance=200)
    return d3.show(filepath='./d3graph_1.html')


<h1>D3Graph: d3_demo</h1>
A more elaborate demonstration of d3graph.

In [118]:
def make_empty_graph(data, explicit_priorities=True, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    animations, df, priorities, priority_collisions, mod_collisions = data
    if anim_kwd != None and not anim_kwd.isspace():
        df = df[df.Animation.str.contains(anim_kwd)]
    if mod_kwd != None and not mod_kwd.isspace():
        df = df[df.Mod.str.contains(mod_kwd)]
    if only_conflicts:
        df = df[df.Priority.in(priority_collisions.keys())]
            
    mods = list(df.sort_values(['Mod'], inplace=False)['Mod'].unique())
    anims = list(df.sort_values(['Animation'], inplace=False).Animation.unique())
    
    if explicit_priorities:
        priorities = list(df.sort_values(['Priority'], inplace=False).Priority.unique())
    else: priorities = []
    
    RED = '#FF0000'
    BLU = '#0000FF'
    GRN = '#00FF00'
    #RED = [1.0,0.0,0.0]
    #BLU = [0.0,255.0,0.0]
    #GRN = [0.0,0.0,255.0]
    #RED = rgb(255,0,0)
    #BLU = rgb(0,255.0)
    #GRN = rgb(0,0,255)
    
    colors = [RED for x in mods] + [BLU for x in priorities] + [GRN for x in anims]
    labels = [x for x in mods] + [x for x in priorities] + [x for x in anims]
    sizes = [6 for x in mods] + [4 for x in priorities] + [6 for x in anims]
    #tooltips = mods + priorities + anims
    
    tooltips = []
    for mod in mods:
        tooltips.append(f'Mod: {mod}')
    for priority in priorities:
        tooltips.append(f'Priority: {priority} <em>TESTTESTTEST</em>')
    for anim in anims:
        tooltips.append(f'<b>Animation</b>: {anim}')
        
    cleaned = [clean(x) for x in mods] + [str(x) for x in priorities] + [clean(x) for x in anims]
    
    df_meta = pd.DataFrame(
        list(zip(tooltips, labels, colors, sizes)), 
        index=cleaned, 
        columns=['Tooltip', 'Label', 'Color', 'Size'])

    df_graph = pd.DataFrame(columns=cleaned, index=cleaned)
    df_graph.fillna(0, inplace=True)
    df_graph.astype(int)
    
    return df_graph, df_meta, mods, anims, priorities

SyntaxError: invalid syntax (1359495189.py, line 8)

In [119]:
def make_connectivity_graph(data, explicit_priorities=False, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    animations, df, priorities, priority_collisions, mod_collisions = data
    if anim_kwd != None and not anim_kwd.isspace():
        df = df[df.Animation.str.contains(anim_kwd)]
    if mod_kwd != None and not mod_kwd.isspace():
        df = df[df.Mod.str.contains(mod_kwd)]
    if only_conflicts:
        df = df[df.Priority.isin(priority_collisions.keys())]

    df_graph, df_meta, mods, anims, priorities = make_empty_graph(df, explicit_priorities)
    print(df_meta.head())
    
    for a in animations:
        if a not in anims: continue

        #print(anim)
        priorities = animations[a]
        animation = clean(a)

        for p in priorities:
            if not only_conflicts or p in priority_collisions.keys():
                mods = priorities[p]
                priority = str(p)

                #row = df_graph.loc[priority]
                if explicit_priorities: df_graph.loc[priority, animation] += 1
                #print(df_graph.loc[priority, animation])

                #source.append(anim)
                #target.append(priority)
                #weight.append(len(mods))

                for m in mods:
                    mod = clean(m)
                    if explicit_priorities: df_graph.loc[mod, priority] += 1
                    df_graph.loc[mod, animation] += 1
    
    return df_graph, df_meta

In [120]:
def d3_demo(data, explicit_priorities=True, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    from d3graph import d3graph
    
    animations, df, priorities, priority_collisions, mod_collisions = data
    df_graph, df_meta = make_connectivity_graph(data, animations, explicit_priorities, anim_kwd, mod_kwd)

    df_graph_head = df_graph.iloc[:,:]
    df_meta_head = df_meta.iloc[:,:]

    tooltips = df_meta_head['Tooltip'].values
    labels = df_meta_head['Label'].values
    colors = df_meta_head['Color'].values
    sizes=df_meta_head['Size'].values

    df_meta_head.head()

    print(df_graph_head.shape)
    print(df_meta_head.shape)
    print(len(tooltips))

    d3 = d3graph()

    # Proces adjmat
    d3.graph(df_graph_head)

    d3.set_edge_properties(edge_distance=200)

    # Make changes in node properties
    d3.set_node_properties(
        tooltip=tooltips, 
        label=labels, 
        color=colors, 
        size=sizes)

    #tooltip = '\nId: ' + adjmat.columns.astype(str) +'\nDegree: ' + df['degree'].astype(str) + '\nLabel: ' + df['label'].values

    #d3.set_node_properties(
    #    color=adjmat.columns.values, tooltip=df_meta['Label'].values, color=df_meta['Color'].values, threshold=1)
    # Plot
    d3.show(filepath='c://temp/')

<h1>BOKEH</h1>
<p>Based on 
<a href='https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html'>
    https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html
</a>
</p>
<p>NO INTERACTIVITY</p>

In [121]:
def bokeh_test(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    import pandas as pd
    import networkx
    import matplotlib.pyplot as plt
    import numpy as np    
    from bokeh.io import output_notebook, show, save
    from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
    from bokeh.plotting import figure
    from bokeh.plotting import from_networkx
    from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
    from bokeh.transform import linear_cmap
    from bokeh.models import PointDrawTool
    from networkx.algorithms import community

    animations, df, priorities, priority_collisions, mod_collisions = data
    
    #Choose colors for node and edge highlighting
    node_highlight_color = 'white'
    edge_highlight_color = 'black'

    output_notebook()   
    
    if anim_kwd != None and not anim_kwd.isspace():
        df = df[df.Animation.str.contains(anim_kwd)]
    if mod_kwd != None and not mod_kwd.isspace():
        df = df[df.Mod.str.contains(mod_kwd)]
    if only_conflicts:
        df = df[df.Priority.isin(priority_collisions.keys())]
    G = networkx.from_pandas_edgelist(df, 'Animation', 'Mod')

    # Set node properties
    degrees = dict(networkx.degree(G))
    node_types = {mod:"Mod" for mod in df.Mod.unique()} |  {anim:'Anim' for anim in df.Animation.unique()}
    node_sizes = {mod:15 for mod in df.Mod.unique()} |  {anim:10 for anim in df.Animation.unique()}
    node_colors = {mod:"red" for mod in df.Mod.unique()} |  {anim:'skyblue' for anim in df.Animation.unique()}
    
    networkx.set_node_attributes(G, name='degree', values=degrees)
    networkx.set_node_attributes(G, name='type', values=node_types)
    networkx.set_node_attributes(G, name='node_size', values=node_sizes)
    networkx.set_node_attributes(G, name='node_color', values=node_colors)
    
    size_by_this_attribute = 'node_size'
    color_by_this_attribute = 'node_color'    

    #Choose a title!
    title = 'DAR Mods and Animations Connectivity'

    #Establish which categories will appear when hovering over each node
    HOVER_TOOLTIPS = [("Type", "@type"), ("Name", "@index"), ("Degree", "@degree")]

    #Create a plot — set dimensions, toolbar, and title
    plot = figure(
        tooltips=HOVER_TOOLTIPS, 
        x_range=Range1d(-10.1, 10.1), 
        y_range=Range1d(-10.1, 10.1), 
        title=title)

    #Create a network graph object with spring layout
    # https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
    network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

    #Set node size and color
    network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
   
    #Set highlighting
    network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
    network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
    #Set edge opacity and width
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
    #Set edge highlight colors
    network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
    network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

    #Highlight nodes and edges
    network_graph.selection_policy = NodesAndLinkedEdges()
    network_graph.inspection_policy = NodesAndLinkedEdges()

    #plot.add_tools(PointDrawTool(renderers=[network_graph.node_renderer], empty_value='black'))

    #Add network graph to the plot
    plot.renderers.append(network_graph)

    show(plot)
    #save(plot, filename=f"{title}.html")

#bokeh_test(df, anim_kwd='jump')

<h1>NetGraph</h1>
<p>Based on 
<a href='https://github.com/paulbrodersen/netgraph'>
    https://github.com/paulbrodersen/netgraph
</a>
</p>
<p>
    NOT YET DEVELOPED -- UNINTERACTIVE IN JUPYTER NOTEBOOKS
</p>

In [122]:
def netgraph_demo(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    %matplotlib notebook
    import matplotlib.pyplot as plt
    import networkx as nx
    from netgraph import Graph, InteractiveGraph

    animations, df, priorities, priority_collisions, mod_collisions = data
    
    # create a random geometric graph and plot it
    g = nx.random_geometric_graph(100, 0.15, seed=42)
    node_positions = nx.get_node_attributes(g, 'pos')
    plot_instance = InteractiveGraph(g,
                          node_layout=node_positions,
                          node_size=1,
                          node_edge_width=0.1,
                          edge_width=0.1)

    # select a random path in the network and plot it
    path = nx.shortest_path(g, 33, 66)

    for node in path:
        plot_instance.node_artists[node].radius = 1.5 * 1e-2
        plot_instance.node_artists[node].set_color('orange')

    for ii, node_1 in enumerate(path[:-1]):
        node_2 = path[ii+1]
        if (node_1, node_2) in plot_instance.edges:
            edge = (node_1, node_2)
        else: # the edge is specified in reverse node order
            edge = (node_2, node_1)
        plot_instance.edge_artists[edge].update_width(0.5 * 1e-2)
        plot_instance.edge_artists[edge].set_color('red')
        plot_instance.edge_artists[edge].set_alpha(1.0)

    return plt.show()
    
#netgraph_demo(df, anim_kwd='idle')

<h1>iPyCytoScape</h1>
<p>Based on:</p>
<p>
<a href='https://hub.gke2.mybinder.org/user/quantstack-ipycytoscape-181mth2e/tree/examples'>
    https://hub.gke2.mybinder.org/user/quantstack-ipycytoscape-181mth2e/tree/examples
</a>
<p>
<a href='https://blog.jupyter.org/interactive-graph-visualization-in-jupyter-with-ipycytoscape-a8828a54ab63'>
    https://blog.jupyter.org/interactive-graph-visualization-in-jupyter-with-ipycytoscape-a8828a54ab63
</a>
</p>
<p>
<a href='https://ipycytoscape.readthedocs.io/en/latest/examples/interaction.html#Reacting-to-User-Interaction-in-the-Kernel'>
    https://ipycytoscape.readthedocs.io/en/latest/examples/interaction.html#Reacting-to-User-Interaction-in-the-Kernel
</a>
</p>
https://ipycytoscape.readthedocs.io/en/master/examples/networkx.html

In [123]:
def ipycytoscape_test(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    import ipycytoscape
    import ipywidgets as widgets
    import networkx as nx
    from random import random

    animations, df, priorities, priority_collisions, mod_collisions = data
    
    class DARNode(ipycytoscape.Node):
        def __init__(self, uid, name, classes=''):
            super().__init__()
            self.data['id'] = uid
            self.classes = classes
            
    G = nx.Graph()
    node_ids = {}
    nodes = {}
    
    for animation in animations:
        if anim_kwd == None or anim_kwd.isspace() or anim_kwd in animation:
            
            node_ids[animation] = clean(animation)
            nodes[animation] = DARNode(clean(animation), animation, 'Animation')
            G.add_node(nodes[animation])

            priorities = animations[animation]
            for priority in priorities:
                if not only_conflicts or priority in priority_collisions.keys():
                    if not priority in node_ids.keys(): node_ids[priority] = priority
                    priority_id = node_ids[priority]
                    #pvnet.add_node(priority_id, label=str(priority), title=str(priority), size=20, color='green', font='120px arial')
                    #pvnet.add_edge(anim_id, priority_id)

                    mods = priorities[priority]
                    for mod in mods:
                        if mod_kwd == None or mod_kwd.isspace() or mod_kwd in mod:
                            if not mod in node_ids.keys(): node_ids[mod] = clean(mod)
                            if not mod in nodes.keys(): 
                                nodes[mod] = DARNode(node_ids[mod], mod, 'Mod')
                                G.add_node(nodes[mod])

                            G.add_edge(nodes[animation], nodes[mod])
    
    #G.add_nodes_from([DARNode(clean(mod), mod, 'mod') for mod in df.Mod.unique()])
    #G.add_nodes([DARNode(clean(anim), anim, 'anim') for anim in df.Animation.unique()])
    
    #G = networkx.from_pandas_edgelist(df, 'Animation', 'Mod')

    
    #G = nx.complete_graph(5)
    #G = nx.newman_watts_strogatz_graph(50, 2, 0.2)
    cyto = ipycytoscape.CytoscapeWidget()
    cyto.graph.add_graph_from_networkx(G)
    
    def log_clicks(node):
        cyto.set_layout(nodeSpacing = 6)
        cyto.set_layout(nodeSpacing = 5)
        #G2 = nx.Graph()
        #G2.add_node("PERTURBER ")
        #cyto.graph.add_graph_from_networkx(G2)
        #cyto.graph.remove_node_by_id("PERTURBER")
        #cyto.set_layout()
        #print(f'clicked: {node}')

    #def log_mouseovers(node):
        #print(f'mouseover: {node}')
    
    cyto.set_layout(
        name = 'cola', 
        nodeSpacing = 5, 
        edgeLengthVal = 45, 
        animate = True, 
        randomize = False, 
        maxSimulationTime = 30000)
    
    cyto.set_style([
                        {
                            'selector': 'node.Animation',
                            'css': {
                                'background-color': 'red'
                            }
                        },
                        {
                            'selector': 'node.anim',
                            'css': {
                                'background-color': 'blue'
                            }
                        }])
    
    cyto.on('node', 'tapdrag', log_clicks)
    display(cyto)
    return G, cyto

#G, cyto = ipycytoscape_test(data, only_conflicts=False, anim_kwd='jump')

<h1>NetworkX: networkx_demo</h1>
<p>Based on 
<a href='https://www.kaggle.com/code/arthurtok/ghastly-network-and-d3-js-force-directed-graphs/notebook'>
    https://www.kaggle.com/code/arthurtok/ghastly-network-and-d3-js-force-directed-graphs/notebook
</a>
</p>

In [124]:
import networkx as nx
G2 = nx.newman_watts_strogatz_graph(50, 2, 0.2, seed=10)
cyto.graph.add_graph_from_networkx(G2)

In [125]:
def networkx_demo(data, only_conflicts=True, anim_kwd=None, mod_kwd=None):
    import matplotlib.pyplot as plt
    import networkx as nx
    import nltk
    from nltk.util import ngrams
    %matplotlib inline

    animations, df, priorities, priority_collisions, mod_collisions = data
    if anim_kwd != None and not anim_kwd.isspace():
        df = df[df.Animation.str.contains(anim_kwd)]
    if mod_kwd != None and not mod_kwd.isspace():
        df = df[df.Mod.str.contains(mod_kwd)]
    if only_conflicts:
        df = df[df.Priority.isin(priority_collisions.keys())]

    g = nx.from_pandas_edgelist(df,source='Animation',target='Mod')
    print(nx.info(g))
    
    plt.figure(figsize=(20, 20))
    
    cmap = plt.cm.coolwarm
    colors = [n for n in range(len(g.nodes()))]
    #k = 0.0319
    k = 0.14
    pos=nx.spring_layout(g, k=k)
    nx.draw_networkx(g, pos, cmap = cmap, node_color=colors, 
                     edge_color='grey', font_size=15, width=2, alpha=1)
    
    #nx.draw_networkx(g,pos, node_size=trigram_df['count'].values*150, cmap = cmap, 
    #                 node_color=colors, edge_color='grey', font_size=15, width=2, alpha=1)
    plt.title("Network diagram of Top 20 Trigrams w/o Stopwords removed attributed to each Author",
             fontsize=18)
    plt.show()


<h1>D3.js: d3_js_demo</h1>
<p>Based on 
<a href='https://www.kaggle.com/code/arthurtok/ghastly-network-and-d3-js-force-directed-graphs/notebook'>
    https://www.kaggle.com/code/arthurtok/ghastly-network-and-d3-js-force-directed-graphs/notebook
</a>
</p>

In [126]:
html_string = """
<!DOCTYPE html>
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 3px;
}

text {
  font-family: sans-serif;
  font-size: 12px;
}

</style>
<svg width="960" height="500"></svg>
"""
js_string="""
 require.config({
    paths: {
        d3: "https://d3js.org/d3.v4.min"
     }
 });

  require(["d3"], function(d3) {

  var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");

var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().distance(170).strength(0.5).id(function(d) { return d.id; }))
    .force("charge", d3.forceManyBody())
    .force("center", d3.forceCenter(width/2 , height/2 ));

d3.json("pcap_export.json", function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
      .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

  var node = svg.append("g")
      .attr("class", "nodes")
    .selectAll("g")
    .data(graph.nodes)
    .enter().append("g")

  var circles = node.append("circle")
      .attr("r", 8)
      .attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  var lables = node.append("text")
      .text(function(d) {
        return d.id;
      })
      .attr('x', 6)
      .attr('y', 3);

  node.append("title")
      .text(function(d) { return d.id; });

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(graph.links);

  function ticked() {
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("transform", function(d) {
          return "translate(" + d.x + "," + d.y + ")";
        })
  }
});

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.9).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}  

  });
 """

def d3_js_demo():
    import IPython
    import json, random
    import IPython.display
    from IPython.core.display import display, HTML, Javascript
    import json, random
    from string import Template
    
    from IPython.display import IFrame
    import ipywidgets as widgets
    
    from IPython.display import display, IFrame

    out = widgets.Output()

    @out.capture()
    def render():
        display(IFrame(src='file.html',width=900, height=600))
    
    #%matplotlib inline
    #return IFrame(src='index2.html', width=700, height=600)
    #%matplotlib inline
    #h = IPython.display.display(HTML(html_string))
    #j = IPython.display.Javascript(js_string)
    #IPython.display.display_javascript(j)


https://gist.github.com/fancellu/2c782394602a93921faff74e594d1bb1

In [145]:
def run_tests():
    data = make_animations_dataframe()    
    pyvis_d3_force(animations, anim_kwd='jump')
    simple_force_graph(animations, anim_kwd='jump')
    d3_demo(animations, df, anim_kwd='jump')
    networkx_demo(df, anim_kwd='jump')
    
data = make_animations_dataframe()    
print_summary(data)

9619 DAR animations found (out of either 16384 or 32768, depending on version)
109 priority collisions were found.
Priority Collisions
├── Collision
│   ├── Of mods
│   │   ├── adri - more cinematic reactions
│   │   └── adri - project impact 2.72
│   └── Of priorities
│       ├── 3890001
│       ├── 3890002
│       ├── 3890003
│       ├── 3890004
│       ├── 3890005
│       ├── 3890006
│       ├── 3890007
│       ├── 3890008
│       ├── 7700011
│       ├── 7700021
│       ├── 7700022
│       ├── 7700023
│       ├── 7700024
│       ├── 7711001
│       ├── 7711002
│       ├── 7721001
│       ├── 7721002
│       ├── 7731001
│       ├── 7731002
│       ├── 7741001
│       ├── 7741002
│       ├── 7751001
│       ├── 7751002
│       ├── 7761001
│       ├── 7761002
│       └── 8813031
├── Collision
│   ├── Of mods
│   │   ├── adxp i mco skyrim martial arts - bryan fury
│   │   ├── adxp i mco skyrim martial arts - kick boxing
│   │   └── adxp i mco skyrim martial arts - muay thai
│   └── Of p

In [150]:
#display_treemap(data, only_conflicts=True)
pyvis_d3_force(data)
#simple_force_graph(data, only_conflicts=True)
#make_pcap_json(data, only_conflicts=True, anim_kwd='jump')
#d3_demo(data)
#bokeh_test(data)

#make_pcap_json(data, only_conflicts=True, anim_kwd='jump')
#d3_js_demo()
#bokeh_test(data, only_conflicts=True, anim_kwd='jump')
#G, cyto = ipycytoscape_test(data, anim_kwd='idle')
#display(cyto)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


IFRAME
<iframe src='http://localhost:8000/index.html', width=700, height=600></iframe>